Adding Wind Pressure Coefficients to Surfaces in an E+ IDF File using Python
==================================================================================================

Introduction
------------

In this tutorial, we will walk you through a Python script designed to add wind pressure coefficients to surfaces in an EnergyPlus IDF file. This can be particularly useful when you're modelling airflow and want to understand how the building's geometry interacts with the wind.

Prerequisites
-------------

Before we begin, ensure you run the following in terminal:

```
conda env create -f EP_environment.yml
conda activate EP
```

The Script
----------

The script you'll be working with uses the EPPY library to interact with EnergyPlus IDF files. We'll start by importing necessary modules, setting paths, and then diving into the core logic of the script.

In [1]:
from update_cp import *

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Adding CP Values to IDF')

# Constants
IDF_PATH = 'data/in.idf'                # Change this!
IDD_PATH = 'idd/Energy+_22_2_0.idd'

# Which CP values to use
CP_PATH = 'data/Cp.csv'                 # Change this!

# Change this as needed
OUTPUT_PATH = 'in_with_cp.idf'          # Change this!

# Translation values from the origin
X_TRANS = 750
Y_TRANS = 900
Z_TRANS = 0

In [3]:
# First, check the paths for the input data and configuration files.
path_checker(IDF_PATH, IDD_PATH, CP_PATH)

# Log the location of the IDD file that will be used.
logger.info(f'Setting IDD file to {IDD_PATH}')

# Set the IDD file for the EPPY library. The IDD file provides the structure and 
# definitions for EnergyPlus input files (IDF). EPPY uses the IDD to understand 
# how to correctly interpret and manipulate IDF files.
IDF.setiddname(IDD_PATH)

# Log the path of the IDF file that will be loaded.
logger.info(f'Loading IDF file from {IDF_PATH}')

# Load the IDF file using EPPY's IDF class.
idf1 = IDF(IDF_PATH)

# Update certain parameters in the IDF. The details of the updates are abstracted 
# in the function and are not evident from the function name.
update_idf_params(idf1)

# Calculate or retrieve wind pressure coefficients for the building model.
# These coefficients represent the wind's effect on different surfaces of the building.
wind_pressure_coefficient_array = add_wind_pressure_coefficients_array(idf1)

# Create a k-d tree and a dataframe for vectorized points. These might be used 
# for efficient spatial searches or operations.
kdtree, coords_df = vectorize_points(CP_PATH,X_TRANS,Y_TRANS,Z_TRANS)

# Fetch building surfaces and fenestration objects (like windows or doors) from the IDF.
building_objects, fenestration_objects = fetch_surfaces(idf1)

# Plot fenestration objects and building surfaces.
plot_objects(fenestration_objects,coords_df)

# Add wind pressure coefficients to the fenestration objects in the IDF.
add_cp_fenestration(idf1, fenestration_objects, wind_pressure_coefficient_array, kdtree=kdtree, coords_df=coords_df)

# Fetch building surfaces that do not have an external node name. 
# This suggests surfaces where the wind might not directly interact, like internal walls.
building_objects_no_external_node_name = get_surface_no_node(idf1, building_objects)

# Add wind pressure coefficients to the building surfaces that do not have external node names.
add_cp_surfaces(idf1, building_objects_no_external_node_name, wind_pressure_coefficient_array, kdtree=kdtree, coords_df=coords_df)

# Update the AirflowNetwork:SimulationControl object in the IDF to specify that 
# wind pressure coefficients will be provided as input rather than calculated by EnergyPlus.
logger.info('Setting AirflowNetwork:SimulationControl object to use input wind pressure coefficients')
idf1.idfobjects['AirflowNetwork:SimulationControl'][0].Wind_Pressure_Coefficient_Type = 'Input'

# Save the modified IDF to a new file named 'in_with_cp.idf'.
logger.info('Saving IDF file with CP')
idf1.saveas(OUTPUT_PATH)

# Log a message indicating that the operation has been completed.
logger.info('Done!')


INFO:Adding CP Values to IDF:Checking paths
INFO:Adding CP Values to IDF:All paths are valid
INFO:Adding CP Values to IDF:Setting IDD file to idd/Energy+_22_2_0.idd
INFO:Adding CP Values to IDF:Loading IDF file from data/in.idf
INFO:Adding CP Values to IDF:Updating IDF to use JtoKWH
INFO:Adding CP Values to IDF:Saving original IDF file
INFO:Adding CP Values to IDF:Adding wind pressure coefficients array
INFO:Adding CP Values to IDF:Vectorizing points from data/Cp.csv
INFO:Adding CP Values to IDF:Translating points in the Cp.csv file by 750, 900 and 0
INFO:Adding CP Values to IDF:Building KDTree
INFO:Adding CP Values to IDF:Fetching surfaces


INFO:Adding CP Values to IDF:Adding CP for fenestration


INFO:Adding CP Values to IDF:Getting surfaces with no node setup


INFO:Adding CP Values to IDF:Adding CP for surfaces


INFO:Adding CP Values to IDF:Setting AirflowNetwork:SimulationControl object to use input wind pressure coefficients
INFO:Adding CP Values to IDF:Saving IDF file with CP


INFO:Adding CP Values to IDF:Done!
